# **Notebook for Home Price Predictions** #


<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ageron/handson-ml/blob/master/01_the_machine_learning_landscape.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## **Setup**

In [3]:
from google.colab import drive # import drive from google colab
 
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
 
drive.mount(ROOT)

/content/drive
Mounted at /content/drive


In [4]:
%cd drive/MyDrive/kaggle-home-prices/

/content/drive/MyDrive/kaggle-home-prices


In [ ]:
!pip install scikit-learn-intelex -q --progress-bar off
!pip install optuna

## **Load Data**

In [6]:
import numpy as np # linear algebra
import pandas as pd 
import os
 
os.chdir("data")

In [7]:
import sklearn
import seaborn as sns
from xgboost import XGBRegressor
# work around for installing sklearnrex
import sys
import os
import site
sys.path.append(os.path.join(os.path.dirname(site.getsitepackages()[0]), "site-packages"))
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [8]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [9]:
from sklearn.model_selection import train_test_split
practice_train_set, practice_test_set = train_test_split(train, test_size=0.2, random_state=42)

## **Transformers**

In [10]:
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn_pandas import CategoricalImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor


class DataFrameSelector(BaseEstimator, TransformerMixin):
  def __init__(self, attribute_names):
    self.attribute_names = attribute_names
  def fit(self, X, y=None):
    return self  
  def transform(self, X):
    return X[self.attribute_names].values  


y = train["SalePrice"].copy()
X = train.drop("SalePrice", axis=1)

num_attribs = list(X.select_dtypes(exclude='object').keys())
cat_attribs = list(X.select_dtypes(include='object').keys())

y_practice = practice_train_set["SalePrice"].copy()
X_practice = practice_train_set.drop("SalePrice", axis=1)
y_practice_test = practice_test_set["SalePrice"].copy()
X_practice_test = practice_test_set.drop("SalePrice", axis=1)

num_pipeline= Pipeline([('selector', DataFrameSelector(num_attribs)), ('imputer', SimpleImputer(strategy='median')), ('std_scaler', StandardScaler()), ('feature_selector', SelectFromModel(RandomForestRegressor(), threshold=-np.inf, max_features=10)), ])
cat_pipeline= Pipeline([('selector', DataFrameSelector(cat_attribs)), ('imputer', CategoricalImputer()), ('one_hot_encoder', OneHotEncoder(sparse=False, handle_unknown='ignore')), ])
full_pipeline = FeatureUnion(transformer_list=[("num_pipeline", num_pipeline), ("cat_pipeline", cat_pipeline),])

X_train = full_pipeline.fit_transform(X_practice, y_practice)
X_test = full_pipeline.transform(X_practice_test)

In [13]:
X = full_pipeline.fit_transform(X, y)

In [14]:
test_X = full_pipeline.transform(test)
test_y = train["SalePrice"].copy()

##**Helper Functions**

###**Generate RMSE**

In [11]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

def get_RMSE(model, y, X):
  preds = model.predict(X)
  lin_mse = mean_squared_error(y, preds)
  lin_rmse = np.sqrt(lin_mse)
  print("RMSE", lin_rmse)
  lin_log_mse = mean_squared_error(np.log(y), np.log(preds))
  lin_log_rmse = np.sqrt(lin_log_mse)
  print("log RMSE", lin_log_rmse)

###**Write to CSV**

In [12]:
import csv

def write_to_csv(predictions, filename):
  # open the file in the write mode
  f = open(filename, 'w+')
  writer = csv.writer(f)
  writer.writerow(["Id", "SalePrice"])
  for count, id in enumerate(test["Id"]):
    writer.writerow([id, predictions[count]])
  f.close()  

##**SupportVectorRegressor**

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_log_error
import optuna
def objective_svr(trial):
    params ={
        'max_iter':trial.suggest_float('max_iter', 2000, 5000),
        'C':  trial.suggest_float('C', 1000, 3000),
    }
    model = LinearSVR(**params).fit(X_train, y_practice)
    y_pred = model.predict(X_test)
    loss = np.sqrt(mean_squared_log_error(y_practice_test, y_pred))
    return loss
svr_study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=123),
                            direction="minimize",
                            pruner=optuna.pruners.HyperbandPruner()) 

[I 2021-09-11 04:41:02,474] A new study created in memory with name: no-name-09f6fe8e-58d8-4b0e-86ec-cb1a113c3fe7


In [ ]:
svr_study.optimize(objective_svr, n_trials=1000)

In [22]:
import pickle
svr_model = LinearSVR(**svr_study.best_params).fit(X, y)
final_pred = svr_model.predict(test_X)
#write_to_csv(final_pred, "optuna_svr_submission.csv")
filename = 'optuna_svr_model.sav'
pickle.dump(svr_model, open(filename, 'wb'))

##**GradientBoostingRegressor**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_log_error
import optuna
def objective_gbr(trial):
    params ={
        'n_estimators': trial.suggest_int('n_estimators', 140, 165),
        'max_features':  trial.suggest_int('max_features', 10, 12),
        'max_depth':  trial.suggest_int('max_depth', 4, 8),
        'learning_rate':  trial.suggest_float('learning_rate', 0.08, 0.3),
    }
    model = GradientBoostingRegressor(**params).fit(X_train, y_practice)
    y_pred = model.predict(X_test)
    loss = np.sqrt(mean_squared_log_error(y_practice_test, y_pred))
    return loss
gbr_study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=123),
                            direction="minimize",
                            pruner=optuna.pruners.HyperbandPruner())

[I 2021-09-10 05:51:06,340] A new study created in memory with name: no-name-17e7052f-5a98-462a-a91e-f4a40b6f12ee


In [ ]:
study.optimize(objective_gbr, n_trials=1000)

[I 2021-09-10 05:51:57,292] Trial 498 finished with value: 0.14514185205798202 and parameters: {'n_estimators': 159, 'max_features': 11, 'max_depth': 5, 'learning_rate': 0.18463995650397402}. Best is trial 326 with value: 0.13883308136251335.
[I 2021-09-10 05:51:57,489] Trial 499 finished with value: 0.145076782637612 and parameters: {'n_estimators': 159, 'max_features': 11, 'max_depth': 5, 'learning_rate': 0.18599293368803219}. Best is trial 326 with value: 0.13883308136251335.
[I 2021-09-10 05:51:57,685] Trial 500 finished with value: 0.1633574276639377 and parameters: {'n_estimators': 159, 'max_features': 11, 'max_depth': 5, 'learning_rate': 0.1826761200769223}. Best is trial 326 with value: 0.13883308136251335.
[I 2021-09-10 05:51:57,893] Trial 501 finished with value: 0.15877354700661248 and parameters: {'n_estimators': 160, 'max_features': 11, 'max_depth': 5, 'learning_rate': 0.1662936863627676}. Best is trial 326 with value: 0.13883308136251335.
[I 2021-09-10 05:51:58,093] Trial

In [28]:
import pickle
from sklearn.ensemble import GradientBoostingRegressor

gbr_model = GradientBoostingRegressor(n_estimators=160,max_features=11,max_depth=5,learning_rate=0.16514352203246263).fit(X, y)
final_pred = gbr_model.predict(test_X)
write_to_csv(final_pred, "optuna_gbr_submission.csv")
filename = 'optuna_gbr_model.sav'
pickle.dump(gbr_model, open(filename, 'wb'))

##**XGBRegressor**

In [45]:
 from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_log_error
import optuna
def objective_xgb(trial):
    params ={
        'n_estimators': trial.suggest_int('n_estimators', 300, 400),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'max_depth': 3,
        'learning_rate':  trial.suggest_float('learning_rate', 0.01, 0.1),
    }
    model = XGBRegressor(**params).fit(X_train, y_practice)
    y_pred = model.predict(X_test)
    loss = np.sqrt(mean_squared_log_error(y_practice_test, y_pred))
    return loss
xgb_study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=123),
                            direction="minimize",
                            pruner=optuna.pruners.HyperbandPruner())

[I 2021-09-11 05:13:54,758] A new study created in memory with name: no-name-1e337b9c-67e7-4af8-ae48-ae37db8a0acc


In [46]:
 xgb_study.optimize(objective_xgb, n_trials=1000)

[05:13:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-09-11 05:13:59,052] Trial 0 finished with value: 0.14410829559423996 and parameters: {'n_estimators': 370, 'subsample': 0.7572278669900758, 'learning_rate': 0.030416630820778286}. Best is trial 0 with value: 0.14410829559423996.


[05:13:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-09-11 05:14:01,662] Trial 1 finished with value: 0.14412218747066396 and parameters: {'n_estimators': 355, 'subsample': 0.8438937939571126, 'learning_rate': 0.04807958141120149}. Best is trial 0 with value: 0.14410829559423996.


[05:14:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[I 2021-09-11 05:38:19,732] Trial 556 finished with value: 0.14071768294251077 and parameters: {'n_estimators': 336, 'subsample': 0.8925758734779643, 'learning_rate': 0.09712566874634465}. Best is trial 206 with value: 0.1365824519832614.


[05:38:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-09-11 05:38:22,481] Trial 557 finished with value: 0.14077217054441188 and parameters: {'n_estimators': 389, 'subsample': 0.8860486417495457, 'learning_rate': 0.09459799435051329}. Best is trial 206 with value: 0.1365824519832614.


[05:38:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


KeyboardInterrupt: ignored

In [41]:
print(xgb_study.best_params)
import pickle
xgb_model = XGBRegressor(**xgb_study.best_params).fit(X, y)
final_pred_xgb = gbr_model.predict(test_X)
write_to_csv(final_pred_xgb, "optuna_xgb_submission.csv")
filename = 'optuna_xgb_model2.sav'
pickle.dump(xgb_model, open(filename, 'wb'))

NameError: ignored

##**StackedModel**##

In [24]:
import joblib
xgb = joblib.load('optuna_xgb_model.sav')
gbr = joblib.load('optuna_gbr_model.sav')
svr = joblib.load('optuna_svr_model.sav')

[04:51:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [31]:
from mlxtend.regressor import StackingCVRegressor

stack_gen = StackingCVRegressor(regressors=(xgb, gbr, svr),
                                meta_regressor=xgb,
                                use_features_in_secondary=True)

In [33]:
stack_gen.fit(X, y)

[05:08:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:08:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:08:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:08:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:08:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:08:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:08:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


StackingCVRegressor(cv=5,
                    meta_regressor=XGBRegressor(base_score=0.5,
                                                booster='gbtree',
                                                colsample_bylevel=1,
                                                colsample_bynode=1,
                                                colsample_bytree=1, gamma=0,
                                                importance_type='gain',
                                                learning_rate=0.05660066846236292,
                                                max_delta_step=0, max_depth=3,
                                                min_child_weight=1, missing=nan,
                                                n_estimators=379, n_jobs=1,
                                                nthread=None,
                                                objective='reg:linear',
                                                random_state=0, reg_a...
                                

In [34]:
xgb_final_preds = xgb.predict(test_X)
gbr_final_preds = gbr.predict(test_X)
svr_final_preds = svr.predict(test_X)
stack_gen_final_preds = stack_gen.predict(test_X)

In [47]:
write_to_csv(stack_gen_final_preds, "stack_gen_submission.csv")

In [39]:
best_preds = np.exp(  (np.log(xgb_final_preds) + np.log(gbr_final_preds) + np.log(svr_final_preds) + 2*np.log(stack_gen_final_preds)) / 4 )

write_to_csv(best_preds, "best?_submission.csv")